# Assignment

Assigment is in the intersection of Named Entity Recognition and Dependency Parsing.

0. Evaluate spaCy NER on CoNLL 2003 data (provided)
    - report token-level performance (per class and total)
        - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
    - report CoNLL chunk-level performance (per class and total);
        - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total  

1. Grouping of Entities.
Write a function to group recognized named entities using `noun_chunks` method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks). Analyze the groups in terms of most frequent combinations (i.e. NER types that go together).

2. One of the possible post-processing steps is to fix segmentation errors.
Write a function that extends the entity span to cover the full noun-compounds. Make use of `compound` dependency relation.

# CoNLL Data
From https://www.clips.uantwerpen.be/conll2003/ner/

The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on four types of named entities: persons, locations, organizations and names of miscellaneous entities that do not belong to the previous three groups. 

The CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase.


In [1]:
import conll
import my_ents

# token, POS tag, syntactic chunk tag (IOB), entity tag (IOB)
test = conll.read_corpus_conll('dataset/train.txt')
test = [my_ents.Sentence(sent) for sent in test if '-DOCSTART-' not in sent[0][0]]
test = test[:100]

# 0. Evaluate spaCy NER on CoNLL 2003 data (provided)
- report token-level performance (per class and total)
    - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
- report CoNLL chunk-level performance (per class and total);
    - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total

Conversion from Ontonotes tags to CoNLL format

In [2]:
# conversion of tags from Ontonotes (spacy) to CoNLL format
def from_spacy_to_conll(predictions_spacy):
    switcher = {
                ' ': '',
                '': '',
                'ORG': '-ORG',
                'PER': '-PER',
                'LOC': '-LOC',
                'PERSON': '-PER',
                'GPE': '-LOC'
            }
    
    # LOC, PER, ORG, MISC
    predictions = []
    
    for sent in predictions_spacy:
        new = []
        
        for ent in sent:
            # merge iob and entity type
            new.append((ent.text, ent.ent_iob_ + switcher.get(ent.ent_type_, '-MISC')))
        
        predictions.append(new)
        
    return predictions

## SpaCy predictions with custom tokenizer
Define custom tokenizer for spacy, otherwise spaCy will tokenize differently from how the CoNLL dataset has been tokenized. This would produce different tokens in output, rendering impossible to compute the accuracy.
Because the prediction of each sentence is indipendent of the other sentences, so I used the Pool function to parallelize the task across multiple processes to performe the computation faster. However, for convinience I used a test set of only 100 samples during development, and that case is faster to not use multi processing.

In [3]:
%%time

import spacy
from multiprocessing import Pool

nlp = spacy.load('en_core_web_sm')

def tokenizer_(sent):
    return spacy.tokens.Doc(nlp.vocab, sent.split())

nlp.tokenizer = tokenizer_

if len(test) > 100:
    with Pool(4) as p:
        # spacy predictions, multi-process
        predictions_spacy = p.map(nlp, [str(sent) for sent in test])
else:
    # spacy predictions
    predictions_spacy = [nlp(str(sent)) for sent in test]

CPU times: user 2.1 s, sys: 352 ms, total: 2.45 s
Wall time: 1.76 s


In [4]:
# convert to NLTK format so that conll.evaluate can be used
predictions = from_spacy_to_conll(predictions_spacy)

predictions[6]

[('He', 'O'),
 ('said', 'O'),
 ('further', 'O'),
 ('scientific', 'O'),
 ('study', 'O'),
 ('was', 'O'),
 ('required', 'O'),
 ('and', 'O'),
 ('if', 'O'),
 ('it', 'O'),
 ('was', 'O'),
 ('found', 'O'),
 ('that', 'O'),
 ('action', 'O'),
 ('was', 'O'),
 ('needed', 'O'),
 ('it', 'O'),
 ('should', 'O'),
 ('be', 'O'),
 ('taken', 'O'),
 ('by', 'O'),
 ('the', 'B-ORG'),
 ('European', 'I-ORG'),
 ('Union', 'I-ORG'),
 ('.', 'O')]

## Token-level accuracy, total and per-class

In [5]:
# organize test data in tuples (entity, tag)
test_set = [[(ent.text, ent.ent_tag) for ent in sent.ents] for sent in test]

test_set[6]

[('He', 'O'),
 ('said', 'O'),
 ('further', 'O'),
 ('scientific', 'O'),
 ('study', 'O'),
 ('was', 'O'),
 ('required', 'O'),
 ('and', 'O'),
 ('if', 'O'),
 ('it', 'O'),
 ('was', 'O'),
 ('found', 'O'),
 ('that', 'O'),
 ('action', 'O'),
 ('was', 'O'),
 ('needed', 'O'),
 ('it', 'O'),
 ('should', 'O'),
 ('be', 'O'),
 ('taken', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('European', 'B-ORG'),
 ('Union', 'I-ORG'),
 ('.', 'O')]

### Total accuracy

In [6]:
def total_accuracy(predictions_labels, labels):
    if len(predictions_labels) != len(labels):
        raise Exception('Prediction labels and test labels have different lenght')
    
    correct = 0
    for i in range(len(predictions_labels)):
        if predictions_labels[i] == labels[i]:
            correct += 1
    
    return correct/len(labels)

pred_labels = [ent[1] for sent in predictions for ent in sent]
test_labels = [ent[1] for sent in test_set for ent in sent]

round(total_accuracy(pred_labels, test_labels), 3)

0.853

### Per-class accuracy

In [7]:
from sklearn.metrics import classification_report
import pandas as pd

per_class_metrics = classification_report(test_labels, pred_labels, output_dict=True)
pd_tbl_class = pd.DataFrame().from_dict(per_class_metrics).transpose()
pd_tbl_class.round(decimals=3)

,precision,recall,f1-score,support
B-LOC,0.945,0.776,0.852,67.000
B-MISC,0.218,0.706,0.333,34.000
B-ORG,0.490,0.558,0.522,43.000
B-PER,0.903,0.903,0.903,31.000
I-LOC,0.500,1.000,0.667,1.000
I-MISC,0.000,0.000,0.000,5.000
I-ORG,0.491,0.711,0.581,38.000
I-PER,0.840,0.913,0.875,23.000
O,0.968,0.874,0.919,1414.000
accuracy,0.853,0.853,0.853,0.853


### Chunk-level accuracy

In [8]:
results = conll.evaluate(test_set, predictions)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
LOC,0.945,0.776,0.852,67
PER,0.903,0.903,0.903,31
ORG,0.429,0.488,0.457,43
MISC,0.200,0.647,0.306,34
total,0.502,0.703,0.586,175


# 1. Grouping of Entities.
Write a function to group recognized named entities using `noun_chunks` method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks). Analyze the groups in terms of most frequent combinations (i.e. NER types that go together).

In [9]:
def is_in_a_group(ent, groups):
    for group in groups:
        for elem in group:
            if elem.i == ent.i:
                return True
    return False


def insert_in_group(ent, groups):
    # search the group containing the previous token in sentence order
    for i, group in enumerate(groups):
        for elem in group:
            if elem.i == ent.i - 1:
                # add the ent in next position
                return groups.insert(i+1, [ent])


def group_by_noun_chunk(sent_doc):
    groups = [[ent for ent in chunk] for chunk in sent_doc.noun_chunks]
    
    for ent in sent_doc:
        # check if ent already in a group
        if not is_in_a_group(ent, groups):
            # insert the missing entity in sentence order
            insert_in_group(ent, groups)
    
    return groups

print(predictions_spacy[0])
group_by_noun_chunk(predictions_spacy[0])

EU rejects German call to boycott British lamb . 


[[EU], [rejects], [German, call], [to], [boycott], [British, lamb], [.]]